In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

from researchgate.researchgate.spiders.paper import PaperSpider

In [2]:
process = CrawlerProcess(
    settings={
        "FEEDS": {"items.json": {"format": "json"}},
        "LOG_ENABLED": False
        #     "LOG_LEVEL": 'INFO',
    }
)
process.crawl(PaperSpider)
process.start()

In [3]:
import json

with open("items.json") as f:
    items = json.load(f)

In [4]:
items[0]

{'id': '323694313',
 'title': 'The Lottery Ticket Hypothesis: Training Pruned Neural Networks',
 'abstract': 'Recent work on neural network pruning indicates that, at training time, neural networks need to be significantly larger in size than is necessary to represent the eventual functions that they learn. This paper articulates a new hypothesis to explain this phenomenon. This conjecture, which we term the "lottery ticket hypothesis," proposes that successful training depends on lucky random initialization of a smaller subcomponent of the network. Larger networks have more of these "lottery tickets," meaning they are more likely to luck out with a subcomponent initialized in a configuration amenable to successful optimization. This paper conducts a series of experiments with XOR and MNIST that support the lottery ticket hypothesis. In particular, we identify these fortuitously-initialized subcomponents by pruning low-magnitude weights from trained networks. We then demonstrate that t

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch(hosts=[{"host": "localhost", "port": 9200}])

In [7]:
es.indices.delete("paper-index", ignore=404)

{'acknowledged': True}

In [8]:
es.indices.create(index="paper-index", ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'paper-index'}

In [14]:
# es.index(index="paper-index", id=items[0]["id"], body=items[0])

{'_index': 'paper-index',
 '_type': '_doc',
 '_id': '323694313',
 '_version': 5,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 2}

In [9]:
from elasticsearch_dsl import (
    Document,
    Date,
    Nested,
    Boolean,
    analyzer,
    InnerDoc,
    Completion,
    Keyword,
    Text,
    Integer,
    Float
)

In [10]:
class Paper(Document):
    title = Text(fields={"raw": Keyword()})
    date = Integer()
    abstract = Text()
    authors = Text()
    references = Text()
    page_rank = Float()

    class Index:
        name = "paper-index"

In [11]:
# create the mappings in Elasticsearch
Paper.init(using=es)

In [12]:
for item in items:
    paper = Paper(meta={"id": item["id"]}, **item)
    paper.save(using=es)

In [15]:
from elasticsearch_dsl import Search

In [16]:
s = Search(using=es, index="paper-index").query("match", title="the lottery")

In [132]:
s = Search(using=es, index="paper-index").query("match", id="323694313")

In [17]:
response = s.execute()

In [18]:
for hit in response:
    print(hit.meta.score, hit.title)

5.3779216 The Lottery Ticket Hypothesis: Training Pruned Neural Networks


In [29]:
# by calling .search we get back a standard Search object
s = Paper.search(using=es)
# the search is already limited to the index and doc_type of our document
s = s.query("match", title="the lottery")

In [30]:
response = s.execute()

In [31]:
for hit in response:
    print(hit.meta.score, hit.title)

5.3779216 The Lottery Ticket Hypothesis: Training Pruned Neural Networks


In [13]:
res = es.get(index="paper-index", id="323694313")

In [14]:
res

{'_index': 'paper-index',
 '_type': '_doc',
 '_id': '323694313',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'id': '323694313',
  'title': 'The Lottery Ticket Hypothesis: Training Pruned Neural Networks',
  'abstract': 'Recent work on neural network pruning indicates that, at training time, neural networks need to be significantly larger in size than is necessary to represent the eventual functions that they learn. This paper articulates a new hypothesis to explain this phenomenon. This conjecture, which we term the "lottery ticket hypothesis," proposes that successful training depends on lucky random initialization of a smaller subcomponent of the network. Larger networks have more of these "lottery tickets," meaning they are more likely to luck out with a subcomponent initialized in a configuration amenable to successful optimization. This paper conducts a series of experiments with XOR and MNIST that support the lottery ticket hypothesis. In part

In [32]:
# by calling .search we get back a standard Search object
s = Paper.search(using=es)

In [34]:
res = s.execute()

In [42]:
all_papers = list(res)

In [43]:
all_ids = sorted(list(map(lambda x: x.id, all_papers)))

In [46]:
import numpy as np

In [64]:
p_matrix = np.zeros((len(all_ids), len(all_ids)))

In [65]:
id_loc = dict()

In [66]:
for index, paper_id in enumerate(all_ids):
    id_loc[paper_id] = index

In [67]:
for index, paper_id in enumerate(all_ids):
    paper =  Paper.get(id=paper_id, using=es)
    for reference_id in paper.references:
        try:
            p_matrix[index, id_loc[reference_id]] = 1
        except KeyError:
            continue

In [68]:
p_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])